![Welcome](https://github.com/Resheras/WoS/blob/main/Welcome_WoS.png?raw=true)

# Welcome from Women on Snowflake!

Set the scene: Data Disparity in Health, example Endometriosis, let's use data driven approach to helping solve this problem! - What better place than Snowflake, The Data Cloud?

## We want to show all the COOL stuff, not just SQL:) 

Lets go!


1. Set up your demo account: https://signup.snowflake.com/ using AWS US West Oregon
2. Create basic entities: database, schema.


In [ ]:
CREATE database WID_EVENT;
CREATE schema WID_DEMO;
USE database WID_EVENT;
USE schema WID_DEMO;

Now we can start using the notebook with this database/schema.

## Agenda - Action item - make more general, not feature specific
1. Using Snowflake Cortex to get Gen AI answer questions for you.
2. Optimizing our time by using Cortex to summarise unstructured data.
3. Build interactive dashboards within your Notebook: creating a Streamlit application with Cortex Search to make quick search against a lot of text.
4. Using build-in Snowflake ML functions to attempt to predict the disease based on symptoms. 


### How about we start with asking Snowflake a question - using CORTEX:

In [ ]:
SELECT SNOWFLAKE.CORTEX.COMPLETE('llama3.2-3b', 'What is endometriosis? Give 1 sentence answer') as answer;

In [ ]:
cell6.to_df().first()

### Let's proceed to learn some more - but we don't want to optimize our time reading through long scientific documents. How about we ask Cortex to summarize lengthy reports? We take WHO endo report and save it as pdf, and let magic happen...

In [ ]:
--https://docs.snowflake.com/en/user-guide/snowflake-cortex/document-ai/tutorials/create-processing-pipelines
CREATE or ALTER stage WID_REPORT
ENCRYPTION = ( TYPE = 'SNOWFLAKE_SSE');
LIST @WID_REPORT;

In [ ]:
--add a file you want to analyze - we are using WHO file  from this page: https://www.who.int/news-room/fact-sheets/detail/endometriosis
LIST @WID_REPORT;

In [ ]:
--https://docs.snowflake.com/en/sql-reference/functions/parse_document-snowflake-cortex
SELECT (
    SNOWFLAKE.CORTEX.PARSE_DOCUMENT(
        '@WID_EVENT.WID_DEMO.WID_REPORT',
        'EndometriosisWHO.pdf',
        {'mode': 'OCR'})
    ) AS OCR;

In [ ]:
CREATE or REPLACE TABLE PARSE_RESULT as
select * from {{cell3}};

In [ ]:
select SNOWFLAKE.CORTEX.SUMMARIZE(
select SUBSTRING(TO_VARCHAR(OCR:content), 0, 32768) from PARSE_RESULT);

In [ ]:
cell19.to_df().first()

### Now let's make sure we can educate ourselves with the current state of research. We will use help of Cortex Search and Streamlit.

In [ ]:
Create or replace table ENDO_STUDIES(
SUMMARY VARCHAR,
STUDY VARCHAR, 
PRIMARY_OUTCOME_MEASURES VARCHAR,
SECONDARY_OUTCOME_MEASURES VARCHAR);

In [ ]:
CREATE or ALTER stage WID_STUDIES
ENCRYPTION = ( TYPE = 'SNOWFLAKE_SSE' );

LIST @WID_STUDIES;
--https://clinicaltrials.gov/search?cond=Endometriosis 

In [ ]:
CREATE OR REPLACE FILE FORMAT my_csv 
TYPE = CSV, 
SKIP_HEADER = 1,
FIELD_OPTIONALLY_ENCLOSED_BY = '"';

In [ ]:
COPY INTO ENDO_STUDIES FROM
(SELECT $6, $2, $10, $11
FROM '@wid_studies/WID_Endo_studies_list_for_Cortex.csv'
(FILE_FORMAT => my_csv)
)
FORCE = TRUE;

In [ ]:
select * from ENDO_STUDIES;

In [ ]:
CREATE OR REPLACE CORTEX SEARCH SERVICE study_search_service
  ON study
  ATTRIBUTES summary
  WAREHOUSE = COMPUTE_WH
  TARGET_LAG = '1 day'
  EMBEDDING_MODEL = 'snowflake-arctic-embed-m-v1.5'
  AS (
    SELECT
        summary,
        study,
        primary_outcome_measures
    FROM ENDO_STUDIES
);

In [ ]:
SELECT PARSE_JSON(
  SNOWFLAKE.CORTEX.SEARCH_PREVIEW(
      'wid_event.wid_demo.study_search_service',
      '{
        "query": "prevention",
        "columns":[
            "summary",
            "study"
        ]
      }'
  )
)['results'] as results;

In [ ]:
# Import python packages
import streamlit as st
from snowflake.core import Root
from snowflake.snowpark.context import get_active_session

# Constants
DB = "WID_EVENT"
SCHEMA = "WID_DEMO"
SERVICE = "study_search_service"
BASE_TABLE = "WID_EVENT.WID_DEMO.ENDO_STUDIES"
#ARRAY_ATTRIBUTES = {"SUMMARY"}


def get_column_specification():
    """
    Returns the name of the search column and a list of the names of the attribute columns
    for the provided cortex search service
    """
    session = get_active_session()
    search_service_result = session.sql(f"DESC CORTEX SEARCH SERVICE {DB}.{SCHEMA}.{SERVICE}").collect()[0]
    st.session_state.attribute_columns = search_service_result.attribute_columns.split(",")
    st.session_state.search_column = search_service_result.search_column
    st.session_state.columns = search_service_result.columns.split(",")

def init_layout():
    st.title("Cortex AI Search")
    st.markdown(f"Querying service: `{DB}.{SCHEMA}.{SERVICE}`".replace('"', ''))

def query_cortex_search_service(query):
    """
    Queries the cortex search service in the session state and returns a list of results
    """
    session = get_active_session()
    cortex_search_service = (
        Root(session)
        .databases[DB]
        .schemas[SCHEMA]
        .cortex_search_services[SERVICE]
    )
    context_documents = cortex_search_service.search(
        query,
        columns=st.session_state.columns,
        limit=st.session_state.limit)
    return context_documents.results



def init_search_input():
    st.session_state.query = st.text_input("Query")

def init_limit_input():
    st.session_state.limit = st.number_input("Limit", min_value=1, value=5)


def display_search_results(results):
    """
    Display the search results in the UI
    """
    st.subheader("Search results")
    for i, result in enumerate(results):
        result = dict(result)
        container = st.expander(f"[Result {i+1}]", expanded=True)

        # Add the result text.
        container.markdown(result[st.session_state.search_column])

        # Add the attributes.
        for column, column_value in sorted(result.items()):
            if column == st.session_state.search_column:
                continue
            container.markdown(f"**{column}**: {column_value}")


def main():
    init_layout()
    get_column_specification()
    init_limit_input()
    init_search_input()

    if not st.session_state.query:
        return
    results = query_cortex_search_service(
        st.session_state.query
    )
    display_search_results(results)


if __name__ == "__main__":
    #st.set_page_config(page_title="Cortex AI Search and Summary", layout="wide")
    main()



### Now let's do some more ML magic! Using Snowflake Native ML CLASSIFICATION, we are going to evaluate the possibility of diagnosis based on the common symptoms. Spoiler alert - it is not an easy task. This is why we are talking about increase awareness and make sure resource are dedicated to this research.

In [ ]:
CREATE stage WID_RAW_DATA;
--add file to stage from data tab https://www.kaggle.com/datasets/michaelanietie/endometriosis-dataset
LIST @WID_RAW_DATA;

In [ ]:
CREATE OR REPLACE SEQUENCE seq_01 START = 1 INCREMENT = 1;

In [ ]:
CREATE OR REPLACE TABLE ENDO_DATA (
ROW_ID NUMBER DEFAULT seq_01.NEXTVAL,
AGE NUMBER,
MENSTRUAL_IRREGULARITY BOOLEAN,
CHRONIC_PAIN_LEVEL NUMBER(38,5),
HORMON_LEVEL_ABNORMALITY BOOLEAN,
INFERTILITY BOOLEAN,
BMI NUMBER(38,5),
DIAGNOSIS BOOLEAN
);

SELECT * from ENDO_DATA;

In [ ]:
COPY INTO ENDO_DATA (AGE,MENSTRUAL_IRREGULARITY,CHRONIC_PAIN_LEVEL,HORMON_LEVEL_ABNORMALITY,INFERTILITY,BMI,DIAGNOSIS)
FROM @WID_RAW_DATA
FILE_FORMAT = (TYPE = CSV SKIP_HEADER = 1);

In [ ]:
SELECT diagnosis, count(*) from ENDO_DATA
group by 1;

In [ ]:
--create training set = 80% of data
CREATE OR REPLACE TABLE ENDO_DATA_TRAINING as
SELECT * FROM ENDO_DATA SAMPLE (80);

SELECT * FROM ENDO_DATA_TRAINING;

In [ ]:
CREATE OR REPLACE TABLE ENDO_DATA_TEST as
SELECT * FROM ENDO_DATA
WHERE ROW_ID NOT IN
(SELECT ROW_ID FROM ENDO_DATA_TRAINING);

SELECT * FROM ENDO_DATA_TEST;

In [ ]:
CREATE OR REPLACE view binary_classification_view AS
    SELECT AGE,MENSTRUAL_IRREGULARITY,CHRONIC_PAIN_LEVEL,HORMON_LEVEL_ABNORMALITY,INFERTILITY,BMI,DIAGNOSIS
FROM ENDO_DATA_TRAINING;

SELECT * FROM binary_classification_view ORDER BY RANDOM(42) LIMIT 5;

CREATE OR REPLACE SNOWFLAKE.ML.CLASSIFICATION model_binary(
    INPUT_DATA => SYSTEM$REFERENCE('view', 'binary_classification_view'),
    TARGET_COLNAME => 'DIAGNOSIS'
);

In [ ]:
CREATE OR REPLACE TABLE PREDICTION_RESULTS as
SELECT *, model_binary!PREDICT(INPUT_DATA => {*})
    AS predictions FROM ENDO_DATA_TEST;

In [ ]:
CALL model_binary!SHOW_FEATURE_IMPORTANCE();
--CALL model_binary!SHOW_EVALUATION_METRICS();
--CALL model_binary!SHOW_GLOBAL_EVALUATION_METRICS();
--CALL model_binary!SHOW_THRESHOLD_METRICS();
--CALL model_binary!SHOW_CONFUSION_MATRIX();

In [ ]:
with a as (SELECT *, TO_BOOLEAN(UPPER(AS_VARCHAR(PREDICTIONS:class))) as prediction
from PREDICTION_RESULTS)
select prediction, diagnosis, prediction = diagnosis as correct_prediction, count(row_id) from a
group by 1,2,3;

As a conclusion, we invite you to go deeper into the possibilities of data platforms, as you can see - even small step can help to tackle the data disparity.